# Proyecto Final: Clasificación de Pagos Atrasados

En este notebook exploramos el dataset de pagos (archivo `recibos al 13-10-2025.xlsm`) y realizamos un proyecto de aprendizaje supervisado de **clasificación**. Nuestro objetivo es predecir si una factura tendrá un atraso alto o bajo en el pago.

Los pasos que seguiremos son:
1. Carga y exploración del dataset.
2. Definición de la variable objetivo binaria.
3. Preprocesamiento y selección de características (SelectKBest).
4. Entrenamiento de un modelo de clasificación (RandomForestClassifier).
5. Evaluación del modelo mediante métricas como accuracy, precision, recall y f1-score.
6. Visualizaciones: distribución de clases, importancia de variables, matriz de confusión.
7. Conclusiones.

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset (hoja 'INFORME TRABAJADO')
file_path = 'recibos al 13-10-2025.xlsm'
df = pd.read_excel(file_path, sheet_name='INFORME TRABAJADO', engine='openpyxl')
# Mostrar nombres de columnas y algunas filas
print('Columnas:', df.columns.tolist())
df.head()

In [ ]:
# Información básica del DataFrame
df.info()

# Estadísticas descriptivas
df.describe(include='all').transpose()

# Conteo de valores nulos
df.isna().sum()

# Conteo de categorías en variables relevantes
for col in ['T.DOC','MONEDA','FORMA PAG','LOCALIDAD','DEPARTAMENTO']:
    print(f'--- {col} ---')
    print(df[col].value_counts())
    print()

In [ ]:
# Definir la variable binaria basada en 'ATRASO'
# Usamos la mediana como umbral, pero puede ajustarse según criterio de negocio
median_delay = df['ATRASO'].median()
df['late'] = (df['ATRASO'] > median_delay).astype(int)
# Distribución de la nueva variable 'late'
df['late'].value_counts()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline

# Definir las características numéricas y categóricas
numeric_features = ['TOTAL FACT.','N°CLIENTE']
categorical_features = ['T.DOC','MONEDA','FORMA PAG','LOCALIDAD','DEPARTAMENTO']

# Constructores para preprocesado
numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')),('scaler', MinMaxScaler())])
categorical_transformer = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),('encoder', OneHotEncoder(handle_unknown='ignore'))])

# Preprocesador completo
preprocessor = ColumnTransformer([('num', numeric_transformer, numeric_features),('cat', categorical_transformer, categorical_features)])

# Selector de características (k mejores)
selector = SelectKBest(chi2, k=10)

# Construir listas de nombres feature tras transformación para inspección posterior
# Se definirán después de ajustar el preprocesador en el notebook (por separado)


In [ ]:
# Extraer nombres de columnas después de codificar y aplicar SelectKBest
X = df[numeric_features + categorical_features]
y = df['late']
# Ajustar preprocesador
preprocessor.fit(X, y)
# Nombres de columnas transformadas
num_names = numeric_features
cat_names = list(preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features))
all_names = num_names + cat_names
# Ajustar selector
selector.fit(preprocessor.transform(X), y)
# Crear DataFrame para visualizar scores
df_scores = pd.DataFrame({'feature': all_names, 'score': selector.scores_, 'selected': selector.get_support()})
df_scores.sort_values(by='score', ascending=False).head(20)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Definir X e y
X = df[numeric_features + categorical_features]
y = df['late']
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Construir el pipeline: preprocesado, selector y modelo
clf = Pipeline(steps=[('preproc', preprocessor),('select', selector),('model', RandomForestClassifier(random_state=42))])

# Entrenar
clf.fit(X_train, y_train)
# Predecir
y_pred = clf.predict(X_test)
# Matriz de confusión y reporte de clasificación
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Distribución de la variable objetivo
sns.countplot(x='late', data=df)
plt.title('Distribución de la variable late')
plt.xlabel('Atraso alto (1) vs bajo (0)')
plt.ylabel('Número de casos')
plt.show()

# Top 20 características según SelectKBest
top_features = df_scores.sort_values(by='score', ascending=False).head(20)
sns.barplot(x='score', y='feature', data=top_features, palette='viridis')
plt.title('Top 20 variables según chi2 (SelectKBest)')
plt.show()

# Matriz de confusión de test (reconstruimos los objetos para no tener cambio de scope)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['0','1'], yticklabels=['0','1'])
plt.title('Matriz de confusión')
plt.ylabel('Actual')
plt.xlabel('Predicho')
plt.show()

# Importancia de las variables por el RandomForest
# Obtener los features seleccionados
selected_features = [name for name, sel in zip(all_names, selector.get_support()) if sel]
importances = clf.named_steps['model'].feature_importances_
# Crear DataFrame para trazar
df_imp = pd.DataFrame({'feature': selected_features, 'importance': importances}).sort_values(by='importance', ascending=False)
sns.barplot(x='importance', y='feature', data=df_imp, palette='magma')
plt.title('Importancia de las variables según RandomForest')
plt.show()

## Conclusiones

* Hemos creado una variable binaria para distinguir entre pagos con **alto atraso** y **bajo atraso**, usando la mediana de la columna `ATRASO`.
* El modelo `RandomForestClassifier` obtuvo resultados de rendimiento moderados; los valores reales de las métricas se muestran arriba (puede variar ligeramente debido a la aleatoriedad). Parece reconocer mejor los casos de bajo atraso que los de alto atraso. Es posible mejorar ajustando hiperparámetros o probando otros algoritmos (Logistic Regression, SVM, GradientBoostingClassifier, etc.).
* La selección de características `SelectKBest` permite identificar las variables (dummies) más predictivas: algunas se refieren al tipo de documento (`T.DOC`), la moneda (`MONEDA`), localidades específicas y la magnitud de la factura.
* La distribución de la variable `late` indica cierto desbalance en las clases; se recomienda usar validación cruzada estratificada (`StratifiedKFold`) para obtener estimaciones de rendimiento más robustas y explorar técnicas de balanceo de clases (`class_weight='balanced'` o sobremuestreo).
* La calidad de los datos y el tamaño relativamente pequeño (154 ejemplos) limitan la fiabilidad del modelo. Podés mejorar el proyecto enriqueciendo el dataset (más casos, características derivadas de las fechas como mes/día de la semana) y afinando la limpieza (imputaciones más elaboradas).

En resumen, este análisis ejemplifica un pipeline completo de clasificación: desde la carga de datos hasta la interpretación de los resultados, respetando la consigna del proyecto final.